<a href="https://colab.research.google.com/github/Kyei-frank/Sentiment-Analysis-Project-Deep-Learning-/blob/main/Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connecting Notebook to Google Drive
This will help us to access training data directly from drive without downloading it, as well as to save our model checkpoints to google drive directly

In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


## Installing Packages

In [3]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.5 MB/s eta 0:00:00


## Importing the Required Libraries

In [5]:
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [6]:
# Disabe W&B
# os.environ["WANDB_DISABLED"] = "true"

## Loading the dataset

In [7]:
# Load the dataset and display some values
train_df = pd.read_csv("data/Train.csv")
test_df = pd.read_csv("data/Test.csv")

# A way to eliminate rows containing NaN values
train_df = train_df[~train_df.isna().any(axis=1)]

In [8]:
# Viewing traain_df
train_df.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [9]:
# Viewing test_df
test_df.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


## Splitting Train data into Train and Evaluation subset

In [10]:
# Split the train data => {train, eval}
train, eval = train_test_split(train_df, test_size=0.2, random_state=12, stratify=train_df['label'])

In [11]:
# Viewing train subset
train.head()

,tweet_id,safe_text,label,agreement
4459,MCX0MAM2,"""<user> #Utah health officials monitoring 200 ...",1.0,0.666667
4398,R82Z7JC3,ICYMI Ohio has one of the lowest measles vacci...,1.0,0.666667
7498,9RX21364,New vaccinations required for Maryland student...,0.0,0.666667
8267,G2JY496X,Arrived at Ben Franklin HS to perform! Let's g...,0.0,1.000000
3670,V072BBSB,Health department offers back-to-school immuni...,1.0,0.666667


In [12]:


# Viewing eval subset
eval.head()

,tweet_id,safe_text,label,agreement
9613,EK3CX4V9,<user> New discovery found that inject measles...,-1.0,0.333333
5434,4SAJAII0,"Tragic situation in #Somalia puts our ""recessi...",0.0,1.000000
1799,P6XER9PE,HB1902 [NEW] Motor vehicles; providing civil i...,0.0,1.000000
2548,LIF0EJLW,Such a well-written article on dangerous trend...,1.0,0.666667
3513,0XHNR5TS,<user> Non vaccinated people can still put chi...,1.0,1.000000


In [13]:
# Checking the shape of train & eval subsets after splitting
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (7999, 4), eval is (2000, 4)


In [14]:
# Saving splitted subsets
train.to_csv("data/train_subset.csv", index=False)
eval.to_csv("data/eval_subset.csv", index=False)

In [15]:
dataset = load_dataset('csv',
                        data_files={'train': 'data/train_subset.csv',
                        'eval': 'data/eval_subset.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4c8a33daa85dfcb5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Tokenizing Datasets

Like any other neural network, the transformers also can’t process the raw input text directly, hence it needs to be preprocessed into a form that the model can make sense of. This process is called tokenization where text input is converted into numbers. To do this we use a tokenizer that does the following

- Splitting the input text into words, subwords, or individual letters that are called tokens.
- Mapping each token with a unique integer.
- Arranging and adding required inputs that are useful to the model.

The preprocessing and tokenization process needs to be done in the same way as when the model was trained.  Since we are using pre-trained models, we need to use the corresponding tokenizer for the model and this can be achieved by using AutoTokenizer class. 

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

### Label Transformation

Now as part of the preprocessing steps, we **will** perform two steps:

- Transform	labels and remove the useless columns
- Tokenize the tweets

In [17]:
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length')

# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/7999 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7999
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

## Defining Training Arguments

In [19]:
from transformers import TrainingArguments

# Configure the trianing parameters like `num_train_epochs`: 
# the number of time the model will repeat the training loop over the dataset
training_args = TrainingArguments("test_trainer", num_train_epochs=5,load_best_model_at_end=True,evaluation_strategy="epoch",save_strategy="epoch",)

## Loading a Pretrained Model

In [20]:
from transformers import AutoModelForSequenceClassification
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [21]:
# Defining training and evaluation dataset to use for training
train_dataset = dataset['train'].shuffle(seed=15)
eval_dataset = dataset['eval'].shuffle(seed=15)

## Model Training

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [23]:
# Launch the learning process: training 
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7999
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5000
  Number of trainable parameters = 108312579


Epoch,Training Loss,Validation Loss
1,0.696700,0.649126


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin


Epoch,Training Loss,Validation Loss
1,0.696700,0.649126
2,0.572300,0.617717
3,0.398000,0.794247
4,0.268800,1.155847
5,0.140300,1.404475


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-3000
Configuration saved in test_trainer/checkpoint-3000/config.json
Model weights saved in test_trainer/checkpoint-3000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-4000
Configuration saved in test_trainer/checkpoint-4000/config.json
Model weights saved in test_trainer/checkpoint-4000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-5000
Configuration saved in test_trainer/checkpoint-5000/config.json
Model weights saved in test_tr

TrainOutput(global_step=5000, training_loss=0.41717887115478514, metrics={'train_runtime': 3878.1181, 'train_samples_per_second': 10.313, 'train_steps_per_second': 1.289, 'total_flos': 1.052322114203136e+16, 'train_loss': 0.41717887115478514, 'epoch': 5.0})

## Model Evaluation

In [24]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-24-75db7450f822>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [26]:
# Launch the final evaluation 
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8


{'eval_loss': 0.6177173852920532,
 'eval_accuracy': 0.7455,
 'eval_runtime': 61.4422,
 'eval_samples_per_second': 32.551,
 'eval_steps_per_second': 4.069}